# Code to verify the data extraction process
Feb 27, 2020

### This code verifies that the second part of the extraction ie. storing information in the .npy is correct
It accomplishes this by 
- Reading the .npy files (x,y,idx) for a few IDs
- Reading the file locations for those files directly and extracting the gif files to get the numpy array
- Comparing the images and labels for these (x,y)


In [62]:
import numpy as np
import pandas as pd


import time
import matplotlib.pyplot as plt

### Steps:
- Getting data directly: 
    - Read autoscan_features.3csv and extract a few IDs and corresponding labels at random.
    - Use summary_label_files.csv to get corresponding file names.
    - Extract the image arrays
- Getting data from .npy files
    - Read the idx .npy file to extract the list of IDs taken before and get the index of numpy arrays
    - Now use these indices to extract labels and images from the other .npy files
- Compare
    - For each ID, compare the contents of image numpy arrays and labels
    - For image arrays, compare all 3 file types: temp,srch,diff
    - Done!

## Read data directly from files

### Get IDs and labels

In [45]:
f2='/global/project/projectdirs/dasrepo/vpa/supernova_cnn/data/gathered_data/summary_label_files.csv'
df=pd.read_csv(f2,sep=',',comment='#')

In [46]:
df.head()

,ID,filename,file path,Label
0,801292,srch801292.gif,/global/project/projectdirs/dasrepo/vpa/supern...,1
1,801292,temp801292.gif,/global/project/projectdirs/dasrepo/vpa/supern...,1
2,801292,diff801292.gif,/global/project/projectdirs/dasrepo/vpa/supern...,1
3,800911,temp800911.gif,/global/project/projectdirs/dasrepo/vpa/supern...,1
4,800911,diff800911.gif,/global/project/projectdirs/dasrepo/vpa/supern...,1


In [87]:
num_samples=20
df=df.sample(frac=num_samples/df.shape[0],random_state=37)
IDs=df.ID.values
labels=df.Label.values
print(IDs.shape,labels.shape)

for idx in IDs[:1]:
    df2=df[df.ID==idx]
    display(df2)
    img_dict={}
    
    ### Get label
    label_lst=df[df.ID==idx].Label.values
    ## Check to ensure all 3 labels are same
    assert all(x==label_lst[0] for x in label_lst), "Labels for temp,srch,diff are not identical %"%(label)
    label=label_lst[0]

    ### Get image arrays
    for prefix in ['diff','temp','srch']: ### The order of files is different here, just to spice it up a bit.        
        fle=prefix+str(idx)+'.gif'
        fname=df2[df2.filename==fle]['file path'].values[0]
        img_dict[prefix]=plt.imread(fname)

    ### Get labels and images from .npy arrays
    


,ID,filename,file path,Label
172494,9280371,diff9280371.gif,/global/project/projectdirs/dasrepo/vpa/supern...,1
172495,9280371,temp9280371.gif,/global/project/projectdirs/dasrepo/vpa/supern...,1
172496,9280371,srch9280371.gif,/global/project/projectdirs/dasrepo/vpa/supern...,1


## Read data from .npy files

In [ ]:
## Read ID array

## Read label array

## Read image array

fname='/global/project/projectdirs/dasrepo/vpa/supernova_cnn/data/gathered_data/temp_data/oldx.npy'
a1=np.load(fname)